<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/openai_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain openai pydantic
!pip install -q asyncio
!pip install -q nest_asyncio
!pip install openai-agents -q

In [ ]:
!pip install colab-env --upgrade -q

In [1]:
import asyncio
import os
import random
from agents import Agent, Runner, handoff
from agents import function_tool as tool
from agents import enable_verbose_stdout_logging

from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

import colab_env


# Verify the OPENAI_API_KEY is set (after colab_env has had a chance to load it)
if os.getenv("OPENAI_API_KEY") is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set. "
                     "Please ensure it's loaded via colab_env (from a .env file) "
                     "or set as a system/shell environment variable.")
else:
    print("OPENAI_API_KEY is successfully loaded.")


# --- Define Pydantic Models for structured outputs/inputs ---

class FlightOption(BaseModel):
    airline: str = Field(description="Name of the airline.")
    flight_number: str = Field(description="Flight number.")
    departure_time: str = Field(description="Local departure time (e.g., '10:30 AM').")
    arrival_time: str = Field(description="Local arrival time (e.g., '2:45 PM').")
    price_usd: float = Field(description="Price of the flight in USD.")
    stops: int = Field(description="Number of stops.")
    duration_hours: float = Field(description="Total travel duration in hours.")

class FlightDetails(BaseModel):
    origin: str = Field(description="Departure airport code (e.g., 'YUL').")
    destination: str = Field(description="Arrival airport code (e.g., 'LHR').")
    departure_date: str = Field(description="Departure date in YYYY-MM-DD format.")
    return_date: Optional[str] = Field(None, description="Return date in YYYY-MM-DD format, if round trip.")
    airlines: Optional[List[str]] = Field(None, description="Preferred airlines.")
    max_price: Optional[float] = Field(None, description="Maximum desired flight price.")
    min_stops: int = Field(0, description="Minimum number of stops desired.")

class HotelOption(BaseModel):
    name: str = Field(description="Name of the hotel.")
    address: str = Field(description="Address of the hotel.")
    star_rating: int = Field(description="Star rating of the hotel (e.g., 3, 4, 5).")
    price_per_night_usd: float = Field(description="Price per night in USD.")
    amenities: List[str] = Field(description="List of key amenities (e.g., 'Free WiFi', 'Pool', 'Restaurant').")
    distance_to_center_km: float = Field(description="Distance to city center in kilometers.")

class HotelDetails(BaseModel):
    location: str = Field(description="City or specific area for the hotel.")
    check_in_date: str = Field(description="Hotel check-in date in YYYY-MM-DD format.")
    check_out_date: str = Field(description="Hotel check-out date in YYYY-MM-DD format.")
    num_guests: int = Field(1, description="Number of guests for the hotel.")
    star_rating: Optional[int] = Field(None, description="Minimum star rating for the hotel.")
    max_price_per_night: Optional[float] = Field(None, description="Maximum desired price per night.")

# --- Define Tools (simulated with richer outputs) ---

@tool
def simulated_flight_search(details: FlightDetails) -> List[FlightOption]:
    """
    Simulates searching for flights based on provided details and returns a list of flight options.
    Returns detailed flight options including airline, flight number, times, price, stops, and duration.
    """
    print(f"\n[FlightSearchAgent] Performing a comprehensive simulated flight search for:")
    print(f"  Origin: {details.origin}, Destination: {details.destination}")
    print(f"  Departure: {details.departure_date}")
    if details.return_date:
        print(f"  Return: {details.return_date}")
    if details.airlines:
        print(f"  Preferred Airlines: {', '.join(details.airlines)}")
    if details.max_price:
        print(f"  Max Price: ${details.max_price}")
    print(f"  Min Stops: {details.min_stops}")

    options = []
    base_price = random.uniform(300, 800)

    options.append(FlightOption(
        airline=random.choice(["Air Canada", "British Airways", "Lufthansa"]),
        flight_number=f"AC{random.randint(100,999)}",
        departure_time="10:00 AM",
        arrival_time="10:30 PM",
        price_usd=round(base_price + random.uniform(50, 150), 2),
        stops=0,
        duration_hours=random.uniform(6.5, 8.0)
    ))

    if details.min_stops <= 1:
        options.append(FlightOption(
            airline=random.choice(["United Airlines", "Delta", "KLM"]),
            flight_number=f"DL{random.randint(1000,1999)}",
            departure_time="08:00 AM",
            arrival_time="09:00 PM",
            price_usd=round(base_price - random.uniform(50, 100), 2),
            stops=1,
            duration_hours=random.uniform(10.0, 14.0)
        ))

    if details.min_stops <= 2:
         options.append(FlightOption(
            airline=random.choice(["Ryanair", "EasyJet", "Wizz Air"]),
            flight_number=f"FR{random.randint(2000,2999)}",
            departure_time="06:00 AM",
            arrival_time="11:00 PM",
            price_usd=round(base_price - random.uniform(100, 200), 2),
            stops=2,
            duration_hours=random.uniform(15.0, 20.0)
        ))

    if details.max_price:
        options = [opt for opt in options if opt.price_usd <= details.max_price]

    options = [opt for opt in options if opt.stops >= details.min_stops]

    if not options:
        print(f"[FlightSearchAgent] No flights found matching criteria.")
        return []

    print(f"[FlightSearchAgent] Found {len(options)} simulated flight options.")
    return options

@tool
def simulated_hotel_search(details: HotelDetails) -> List[HotelOption]:
    """
    Simulates searching for hotels based on provided details and returns a list of hotel options.
    Returns detailed hotel options including name, address, star rating, price, amenities, and distance.
    """
    print(f"\n[HotelBookingAgent] Performing a comprehensive simulated hotel search for:")
    print(f"  Location: {details.location}, Check-in: {details.check_in_date}, Check-out: {details.check_out_date}")
    print(f"  Guests: {details.num_guests}")
    if details.star_rating:
        print(f"  Min Star Rating: {details.star_rating}")
    if details.max_price_per_night:
        print(f"  Max Price Per Night: ${details.max_price_per_night}")

    options = []
    base_price = random.uniform(100, 300)

    if not details.star_rating or details.star_rating <= 5:
        options.append(HotelOption(
            name="Grand Lux Hotel",
            address=f"{details.location} Downtown",
            star_rating=5,
            price_per_night_usd=round(base_price * 1.5 + random.uniform(50,100), 2),
            amenities=["Free WiFi", "Pool", "Spa", "Fine Dining", "Concierge"],
            distance_to_center_km=random.uniform(0.5, 2.0)
        ))

    if not details.star_rating or details.star_rating <= 4:
        options.append(HotelOption(
            name="City Comfort Inn",
            address=f"{details.location} Central",
            star_rating=4,
            price_per_night_usd=round(base_price + random.uniform(20,50), 2),
            amenities=["Free WiFi", "Breakfast", "Gym", "Business Center"],
            distance_to_center_km=random.uniform(1.0, 3.5)
        ))

    if not details.star_rating or details.star_rating <= 3:
        options.append(HotelOption(
            name="Value Stay Hostel",
            address=f"{details.location} near Station",
            star_rating=3,
            price_per_night_usd=round(base_price * 0.7 + random.uniform(0,20), 2),
            amenities=["Free WiFi", "Shared Kitchen", "24-hour reception"],
            distance_to_center_km=random.uniform(2.0, 5.0)
        ))

    if details.star_rating:
        options = [opt for opt in options if opt.star_rating >= details.star_rating]

    if details.max_price_per_night:
        options = [opt for opt in options if opt.price_per_night_usd <= details.max_price_per_night]

    if not options:
        print(f"[HotelBookingAgent] No hotels found matching criteria.")
        return []

    print(f"[HotelBookingAgent] Found {len(options)} simulated hotel options.")
    return options

# --- Define Agents ---

flight_search_agent = Agent(
    name="FlightSearchAgent",
    instructions=(
        "You are a specialized AI assistant for finding flight information. "
        "Your goal is to extract all necessary details (origin, destination, departure date, "
        "return date, preferred airlines, maximum price, minimum stops) from the user's request "
        "and use the 'simulated_flight_search' tool to find flights. "
        "When using the tool, ensure you extract all available parameters from the user's request, "
        "such as 'min_stops' or 'max_price'. "
        "Once you receive the list of flight options from the tool, carefully analyze them. "
        "Then, provide a concise and helpful summary of the BEST 2-3 options to the user, highlighting "
        "key details like airline, exact price, number of stops, and total duration. "
        "Prioritize direct flights and shorter durations if prices are comparable. "
        "If no flights are found, inform the user clearly and suggest adjusting criteria (e.g., dates, price). "
        "If you don't have enough information to search, ask clarifying questions, "
        "e.g., 'What are your desired departure and return dates for your flight from [Origin] to [Destination]?'"
    ),
    model="gpt-4o",
    tools=[simulated_flight_search],
)

hotel_booking_agent = Agent(
    name="HotelBookingAgent",
    instructions=(
        "You are a specialized AI assistant for finding hotel information. "
        "Your goal is to extract all necessary details (location, check-in date, check-out date, "
        "number of guests, minimum star rating, maximum price per night) from the user's request "
        "and use the 'simulated_hotel_search' tool to find hotels. "
        "When using the tool, ensure you extract all available parameters from the user's request, "
        "such as 'num_guests' or 'max_price_per_night'. "
        "Once you receive the list of hotel options from the tool, carefully analyze them. "
        "Then, provide a concise and helpful summary of the TOP 2-3 hotel options to the user, highlighting "
        "key details like hotel name, star rating, price per night, and notable amenities. "
        "Prioritize hotels with higher star ratings and closer proximity to the city center if prices are comparable. "
        "If no hotels are found, inform the user clearly and suggest adjusting criteria (e.g., dates, rating, price). "
        "If you don't have enough information to search, ask clarifying questions, "
        "e.g., 'What city are you looking for a hotel in, and what are your check-in and check-out dates?'"
    ),
    model="gpt-4o",
    tools=[simulated_hotel_search],
)

trip_planner_agent = Agent(
    name="TripPlannerAgent",
    instructions=(
        "You are a helpful AI assistant for planning trips. "
        "Your primary role is to understand the user's overall request and hand it off "
        "to the most appropriate specialized agent (FlightSearchAgent or HotelBookingAgent) "
        "if the intent is clearly focused on either flights or hotels and you have enough information. "
        "If the request is ambiguous, involves both flights and hotels, or requires more information, "
        "you should ask clarifying questions to determine the next best step. "
        "For example, if the user asks for 'a trip to Paris', ask if they need flights, hotels, or both, "
        "and for what exact dates (departure, return, check-in, check-out). "
        "If the user simply greets you or asks a general question not related to a specific search, "
        "respond politely and ask how you can help with their trip. "
        "Be proactive in gathering necessary details for the specialized agents. "
        "You should only hand off to an agent if their specific tool (flight search or hotel search) "
        "is clearly applicable and you have enough parameters to make a reasonable call. "
        "Your responses should always be friendly and assist the user in their trip planning journey."
    ),
    model="gpt-4o",
    handoffs=[
        handoff(flight_search_agent),
        handoff(hotel_booking_agent),
    ]
)

# --- Run the Multi-Agent System ---

async def run_multi_agent_system(query: str):
    print(f"User Query: {query}\n")
    runner = Runner()
    result = await runner.run(trip_planner_agent, query)
    print(f"\nFinal Result from Multi-Agent System:\n{result.final_output}")
    print("\n" + "="*50 + "\n") # Separator for multiple runs

# --- Main execution block for Jupyter/Colab and standalone scripts ---

# Enable verbose logging for the agents library to see trace information in console
# This will print trace steps directly to your console/notebook output.
enable_verbose_stdout_logging()

async def main_test_scenarios():
    print("--- Multi-Agent Flight Planning (GPT-4o Edition) ---")
    await run_multi_agent_system("I need to find a flight from Montreal to London for September 15th, 2025, returning on September 22nd.")
    await run_multi_agent_system("I'm looking for a hotel in Paris, France for December 1st to December 5th, 2025, for 2 people with at least 4 stars and max 200 per night.")
    await run_multi_agent_system("Hello there!")
    await run_multi_agent_system("I want to plan a trip next summer.")
    await run_multi_agent_system("Find me a cheap flight to Tokyo in October 2025, with no more than one stop.")
    await run_multi_agent_system("I need a hotel in NYC for 3 nights for 1 person, checking in on July 10th, 2025.")
    await run_multi_agent_system("Plan a vacation for me to Rome for 5 days in November 2025.")


if __name__ == "__main__":
    # Diagnostic for 'tool' (will always print, as it's outside async execution logic)
    print(f"Type of imported 'tool': {type(tool)}")
    print(f"Is 'tool' callable? {callable(tool)}")

    try:
        import nest_asyncio
        nest_asyncio.apply()
        print("Running in existing event loop (likely Jupyter/Colab using nest_asyncio)...")
        asyncio.run(main_test_scenarios())
    except ImportError:
        print("`nest_asyncio` not found. Attempting standard asyncio.run()...")
        try:
            asyncio.run(main_test_scenarios())
        except RuntimeError as e:
            print(f"An asyncio RuntimeError occurred without nest_asyncio: {e}")
            print("If you are in Jupyter/Colab, please install `nest_asyncio` (`pip install nest_asyncio`)")
            print("and restart your kernel, then run this code again.")
            print("Otherwise, ensure your script is not being run from an already active event loop.")
    except Exception as e:
        print(f"An unexpected error occurred during execution: {e}")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
OPENAI_API_KEY is successfully loaded.
Type of imported 'tool': <class 'function'>
Is 'tool' callable? True
Running in existing event loop (likely Jupyter/Colab using nest_asyncio)...
--- Multi-Agent Flight Planning (GPT-4o Edition) ---
User Query: I need to find a flight from Montreal to London for September 15th, 2025, returning on September 22nd.

Creating trace Agent workflow with id trace_d6578e3a7b3d416095a1656111fd681c


DEBUG:openai.agents:Creating trace Agent workflow with id trace_d6578e3a7b3d416095a1656111fd681c


Setting current trace: trace_d6578e3a7b3d416095a1656111fd681c


DEBUG:openai.agents:Setting current trace: trace_d6578e3a7b3d416095a1656111fd681c


Creating span <agents.tracing.span_data.AgentSpanData object at 0x7c247240e810> with id None


...

LLM resp:
[
  {
    "id": "msg_68607edddf48819396f36b85bb6c5f720ccbe637892a32a7",
    "content": [
      {
        "annotations": [],
        "text": "That sounds like a wonderful trip! To help plan your vacation to Rome, could you please let me know if you need assistance with booking flights, hotels, or both? Additionally, if you have specific dates in mind for November 2025, that would be helpful.",
        "type": "output_text",
        "logprobs": []
      }
    ],
    "role": "assistant",
    "status": "completed",
    "type": "message"
  }
]



DEBUG:openai.agents:LLM resp:
[
  {
    "id": "msg_68607edddf48819396f36b85bb6c5f720ccbe637892a32a7",
    "content": [
      {
        "annotations": [],
        "text": "That sounds like a wonderful trip! To help plan your vacation to Rome, could you please let me know if you need assistance with booking flights, hotels, or both? Additionally, if you have specific dates in mind for November 2025, that would be helpful.",
        "type": "output_text",
        "logprobs": []
      }
    ],
    "role": "assistant",
    "status": "completed",
    "type": "message"
  }
]



Resetting current trace


DEBUG:openai.agents:Resetting current trace



Final Result from Multi-Agent System:
That sounds like a wonderful trip! To help plan your vacation to Rome, could you please let me know if you need assistance with booking flights, hotels, or both? Additionally, if you have specific dates in mind for November 2025, that would be helpful.


